In [ ]:
import yfinance as yf
import pandas as pd
# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'Asset_and_Accrual_Detail_CM.csv'

# Read the Excel file
df = pd.read_csv(file_path)

df = df[['Ticker','Security Description 1','Country Code','Local Price','ISIN','Acct Base Currency Code','Issue Currency Code','Segment']]
df = df[df['Segment'] == 'EQUITIES']


: 

In [ ]:
df.head()

: 

## Reading the Asset and Accrual Detail file

In [ ]:
import math

# Iterate over rows and only process those with a valid ticker
for index, row in df[df['Ticker'].notna()].iterrows():
    print(f"Index: {index}, {row['Base Price']}, {row['ISIN']}, {row['Ticker']}")

: 

In [ ]:
df.to_csv('Asset_and_Accrual_Detail_CM with difference.csv', index=False)

### Ticker Modification Settings

In [ ]:
country_mapping = {
    'CA': 'TO',  # Canada
    'HK': 'HK',  # Hong Kong
    # 'US': '',    # United States
    # 'JP': 'T',   # Japan
    # 'GB': 'L',   # United Kingdom
    # 'DE': 'DE',  # Germany
    # 'FR': 'PA',  # France
    # 'IT': 'MI',  # Italy
    # 'ES': 'MC',  # Spain
    # 'AU': 'AX',  # Australia
    # 'IN': 'BO',  # India (Bombay Stock Exchange)
    # 'CN': 'SS',  # China (Shanghai Stock Exchange)
    # 'KR': 'KS',  # South Korea
    # 'SG': 'SI',  # Singapore
    # 'TW': 'TW',  # Taiwan
    # 'BR': 'SA',  # Brazil
    # 'MX': 'MX',  # Mexico
    # 'ZA': 'JO',  # South Africa
    # 'IR' : ""
}

In [ ]:
modifications = {
    '/': '-',
    '1' : '0001'
}

### How ticker should be modified

In [ ]:
for index, row in df.iterrows():
    print(f"Index: {index}, {row['Local Price']}, {row['ISIN']}, {row['Ticker']}")
    if pd.isna(row['Ticker']):
        print(f"Index: {index}, Ticker is NaN")
    else:
        ticker = row['Ticker']
        # Check if any character in the ticker needs modification
        for key, value in modifications.items():
            if key in row['Ticker']:
                ticker = ticker.replace(key, value)
                # print(f"Modified Ticker: {ticker}")
        
    row['Modified Ticker'] = ticker
    try:
        ticker_modification = country_mapping[row['Country Code']]
        # print(ticker_modification)
    except:
        print('No modification')
        ticker_modification = ""
    print(f'final ticker {ticker +"."+ ticker_modification}')

In [ ]:
df.head()

In [ ]:
yesterday = pd.Timestamp.today() - pd.Timedelta(days=1)
def get_current_price(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        current_price = ticker.info.get('currentPrice')
        return current_price
    except Exception as e:
        # print(f"Error retrieving current price for {ticker_symbol}: {e}")
        return None
    
def get_yesterday_price(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        # Get historical data for the past two days with daily intervals
        historical_data = ticker.history(period='5d', interval='1d')

        # Get yesterday's close price
        # yesterday_close = historical_data['Close'].iloc[-2]
        # Get yesterday's date
        # yesterday = pd.Timestamp.today() - pd.Timedelta(days=1)
        # Get the close price for yesterday
        yesterday_close = historical_data.loc[yesterday.strftime('%Y-%m-%d')]['Close']
        return yesterday_close
    except Exception as e:
        return None
        # print(f"Yesterday's close price: {yesterday_close}")

In [ ]:
# Iterate through each row and modify the ticker value if necessary
for index, row in df.iterrows():
    print(f"Index: {index}, {row['Local Price']}, {row['ISIN']}, {row['Ticker']}")
    
    if pd.isna(row['Ticker']):
        print(f"Index: {index}, Ticker is NaN")
        ticker = None
    else:
        ticker = row['Ticker']
        # Check if any character in the ticker needs modification
        for key, value in modifications.items():
            if key in row['Ticker']:
                ticker = ticker.replace(key, value)
                # print(f"Modified Ticker: {ticker}")
    
        try:
            ticker_modification = "."+country_mapping[row['Country Code']]
            # print(ticker_modification)
        except KeyError:
            # print('No modification')
            ticker_modification = ""
        
        final_ticker = ticker+ticker_modification
        # print(f'Final Ticker: {final_ticker}')
        df.at[index, 'Final Ticker'] = final_ticker
        
        try:
            current_price = get_yesterday_price(final_ticker)
        except Exception as e:
            print(f"Error retrieving current price for {row['Ticker']}: {e}")
            current_price = None
        else:
            if current_price is not None:
                difference = float(current_price) - float(row['Local Price'])
                print(row['Ticker'], row['Local Price'], current_price, difference)
                df.at[index, 'Difference'] = difference
                df.at[index, 'Yahoo Finance'] = current_price
            else:
                df.at[index, 'Difference'] = None
        
# df['Yahoo Finance'] = df['Yahoo Finance'].round(3)
# df['Difference'] = df['Difference'].round(3)        
#     # Save the DataFrame to a CSV file
# df.to_csv('modi`fied_tickers.csv', index=False)

# print("The final tickers have been saved to modified_tickers.csv")

### Rounding the Results before saving

In [ ]:

df['Yahoo Finance'] = df['Yahoo Finance'].round(3)
df['Difference'] = df['Difference'].round(3)        
    # Save the DataFrame to a CSV file
df.to_csv('modified_tickers.csv', index=False)